<a href="https://colab.research.google.com/github/Dhanashree-31/BrainTumorDetection/blob/main/BrainTumorDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

IMG_SIZE = 150
data = []
labels = []

base_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/archive/brain_tumor_dataset'

for category in ['yes', 'no']:
    path = os.path.join(base_path, category)
    label = 1 if category == 'yes' else 0
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img_array is not None:
            resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            data.append(resized_array)
            labels.append(label)




In [3]:
data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=32)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 617ms/step - accuracy: 0.5803 - loss: 0.9684 - val_accuracy: 0.7143 - val_loss: 0.6009
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7500 - loss: 0.5690 - val_accuracy: 0.7619 - val_loss: 0.5588
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7822 - loss: 0.4878 - val_accuracy: 0.7619 - val_loss: 0.5806
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8252 - loss: 0.4377 - val_accuracy: 0.7619 - val_loss: 0.5443
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8368 - loss: 0.3847 - val_accuracy: 0.7619 - val_loss: 0.5770
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8512 - loss: 0.3835 - val_accuracy: 0.7619 - val_loss: 0.4945
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8914 - loss: 0.3075 - val_accuracy: 0.8095 - val_loss: 0.4427
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9474 - loss: 0.2076 - val_accuracy: 0.8095 - val_loss: 0.3094

In [12]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step - accuracy: 0.9060 - loss: 0.2218
Test Accuracy: 92.16%


In [13]:
model.save('brain_tumor_cnn_model.h5')

In [16]:
from tensorflow.keras.models import load_model

model = load_model('brain_tumor_cnn_model.h5')
img = cv2.imread('demo2.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).reshape(1, IMG_SIZE, IMG_SIZE, 1) / 255.0

prediction = model.predict(img)
print("Tumor Detected" if prediction[0][0] > 0.5 else "No Tumor")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
Tumor Detected


In [18]:
import tensorflow as tf

model = tf.keras.models.load_model('brain_tumor_cnn_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpy5bseacl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134501195049360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134501195045328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134501195047056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134501195046480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134501195048784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134501195047440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134501195048016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134501195050128: TensorSpec(shape=(), dtype=tf.resource, name=None)
